In [2]:
from myosuite.utils import gym
import mujoco
import mujoco.viewer
import numpy as np
import time


MyoSuite:> Registering Myo Envs


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/workspace/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/workspace/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_sp

In [3]:
env_name = "myoChallengeTableTennisP2-v0"
print(f"Loading {env_name}...")
env = gym.make(env_name)
env.reset()

/workspace/myochallenge_2025eval/.venv/lib/python3.11/site-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


Loading myoChallengeTableTennisP2-v0...


    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


(array([ 2.0205e+00, -6.6971e-04,  9.5000e-01, -4.2050e-01,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -9.7163e-02,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  1.3680e-01,  1.4842e-01,  7.0695e-01,
        -7.4050e-01, -3.4323e-01,  8.0550e-01, -1.0284e-01, -8.2880e-02,
        -7.3042e-01,  6.3200e-02,  7.0000e-01,  7.5030e-02, -2.9673e-01,
         7.2266e-01, -1.3614e-01,  2.6707e-01,  3.5348e-01,  6.5982e-01,
        -1.5184e-01,  4.2417e-01,  5.1843e-01,  9.1904e-01, -2.0944e-01,
         2.5921e-01,  5.1057e-01,  7.9335e-01, -2.0420e-01,  2.2780e-01,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0

In [5]:
_, base_reward, terminated, truncated, info = env.step(env.action_space.sample())

In [6]:
info['rwd_dict']

OrderedDict([('reach_dist', array(0.0484)),
             ('palm_dist', array(0.6062)),
             ('paddle_quat', array(0.3828)),
             ('torso_up', 0.9722480788327826),
             ('act_reg', array(-0.0157)),
             ('sparse', False),
             ('solved', array(False)),
             ('done', array(0)),
             ('dense', array(3.3568))])